# IMDB 영화 리뷰 감성 분석 - Conv1D

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
import warnings
warnings.filterwarnings('ignore')

### Conv1D로 IMDB 리뷰 감성 분석
- 단어 빈도수: 10,000 (총 88,584)
- 문장의 단어수: 500 (최대 2,494)
- test data 중 40%(10,000개)는 검증용으로

In [2]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
X_train.shape, X_test.shape, y_train.shape

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [6]:
from sklearn.model_selection import train_test_split
X_test, X_valid, y_test, y_valid = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.4, random_state=seed
)
X_test.shape, X_valid.shape, y_test.shape, y_valid.shape

((15000, 500), (10000, 500), (15000,), (10000,))

### Case 1) Conv1D x 2, MaxPooling1D x 2, Dropout, GlobalMaxPooling1D
- embedding dim: 100

In [9]:
model1 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 494, 64)           44864     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 70, 64)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 13, 64)           0         
 1D)                                                  

In [10]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [11]:
hist1 = model1.fit(
    X_train, y_train, epochs=30, batch_size=128,
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
196/196 [==============================] - ETA: 0s - loss: 0.5294 - accuracy: 0.7134
Epoch 00001: val_loss improved from inf to 0.36638, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 16s 20ms/step - loss: 0.5294 - accuracy: 0.7134 - val_loss: 0.3664 - val_accuracy: 0.8371
Epoch 2/30
193/196 [============================>.] - ETA: 0s - loss: 0.2768 - accuracy: 0.8876
Epoch 00002: val_loss improved from 0.36638 to 0.31601, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 4s 19ms/step - loss: 0.2769 - accuracy: 0.8876 - val_loss: 0.3160 - val_accuracy: 0.8649
Epoch 3/30
193/196 [============================>.] - ETA: 0s - loss: 0.2034 - accuracy: 0.9220
Epoch 00003: val_loss did not improve from 0.31601
196/196 [==============================] - 4s 18ms/step - loss: 0.2034 - accuracy: 0.9218 - val_loss: 0.3324 - val_accuracy: 0.8634
Epoch 4/30
196/196 [==============================] - ETA: 0s - loss: 0.1493 - accur

In [12]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 4ms/step - loss: 0.3143 - accuracy: 0.8660


[0.31431230902671814, 0.8659999966621399]

### Case 2) Conv1D + LSTM

In [14]:
from tensorflow.keras.layers import LSTM

In [15]:
model2 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_3 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                      

In [16]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [18]:
hist2 = model2.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
195/196 [============================>.] - ETA: 0s - loss: 0.4288 - accuracy: 0.7814
Epoch 00001: val_loss improved from inf to 0.29146, saving model to best-imdb-conv1d-lstm.h5
196/196 [==============================] - 10s 29ms/step - loss: 0.4287 - accuracy: 0.7814 - val_loss: 0.2915 - val_accuracy: 0.8766
Epoch 2/30
195/196 [============================>.] - ETA: 0s - loss: 0.2151 - accuracy: 0.9178
Epoch 00002: val_loss improved from 0.29146 to 0.27755, saving model to best-imdb-conv1d-lstm.h5
196/196 [==============================] - 5s 26ms/step - loss: 0.2151 - accuracy: 0.9179 - val_loss: 0.2776 - val_accuracy: 0.8818
Epoch 3/30
196/196 [==============================] - ETA: 0s - loss: 0.1605 - accuracy: 0.9422
Epoch 00003: val_loss did not improve from 0.27755
196/196 [==============================] - 5s 25ms/step - loss: 0.1605 - accuracy: 0.9422 - val_loss: 0.3336 - val_accuracy: 0.8737
Epoch 4/30
196/196 [==============================] - ETA: 0s - loss: 0.12

In [19]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 4s 7ms/step - loss: 0.2778 - accuracy: 0.8843


[0.27778348326683044, 0.8842666745185852]

### Case 3) Conv1D + Dense

In [20]:
model3 = Sequential([ 
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),  
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_4 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_6 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 100)              

In [21]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-fcn.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [22]:
hist3 = model3.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
192/196 [============================>.] - ETA: 0s - loss: 0.5607 - accuracy: 0.6928
Epoch 00001: val_loss improved from inf to 0.35076, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 4s 18ms/step - loss: 0.5583 - accuracy: 0.6943 - val_loss: 0.3508 - val_accuracy: 0.8476
Epoch 2/30
193/196 [============================>.] - ETA: 0s - loss: 0.2912 - accuracy: 0.8766
Epoch 00002: val_loss improved from 0.35076 to 0.27099, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 3s 17ms/step - loss: 0.2913 - accuracy: 0.8765 - val_loss: 0.2710 - val_accuracy: 0.8849
Epoch 3/30
195/196 [============================>.] - ETA: 0s - loss: 0.1994 - accuracy: 0.9219
Epoch 00003: val_loss improved from 0.27099 to 0.25656, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 3s 17ms/step - loss: 0.1999 - accuracy: 0.9217 - val_loss: 0.2566 - val_accuracy: 0.8952
Epoch 4/30
193/196 [============

In [23]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 1s 3ms/step - loss: 0.2618 - accuracy: 0.8925


[0.26179200410842896, 0.8925333619117737]